In [78]:
# import library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

### Kombinasi 9 :
* Delete Duplicate
* Delete Null
* Outlier capping winsorize sisi kanan
* Encoding 
* Standard scaler
* Feature selection -> K-Best

In [79]:
df = pd.read_csv('../../Without Feature Selection/UFC_kombinasi9_all_features.csv')

In [80]:
X = df.drop(['B_Reach_cms'], axis=1)
y = df['B_Reach_cms']

In [81]:
df_test = pd.read_csv('../../regression_kaggle/UFC_kombinasi9_all_features.csv')
df_test = df_test.drop(['B_Reach_cms'], axis=1, errors='ignore')
df_test_id = df_test['id']
df_test = df_test.drop(['id'], axis=1, errors='ignore')
# Get the common columns between df and df_test
common_columns = list(set(X.columns) & set(df_test.columns))
# Update df_test to only include the common columns
df_test = df_test[common_columns]
X = X[common_columns]

In [82]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [83]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from keras import backend as K

# Define R-squared metric
def r_squared(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return (1 - SS_res/(SS_tot + K.epsilon()))

# Define the model architecture
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(X.shape[1],)))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(256, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))
model.add(Dense(1))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=[r_squared])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=15)

# Train the model
model.fit(X, y, epochs=300, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

# Evaluate the model
loss, r2_score = model.evaluate(X, y)
print("R-squared:", r2_score)

Epoch 1/300
87/87 [==============================] - 1s 8ms/step - loss: 33321.6680 - r_squared: -416.6909 - val_loss: 32983.5508 - val_r_squared: -407.5216
Epoch 2/300
87/87 [==============================] - 0s 5ms/step - loss: 32389.0234 - r_squared: -402.7643 - val_loss: 31636.3555 - val_r_squared: -390.5747
Epoch 3/300
87/87 [==============================] - 0s 6ms/step - loss: 30234.2676 - r_squared: -385.4903 - val_loss: 29018.2188 - val_r_squared: -358.1138
Epoch 4/300
87/87 [==============================] - 1s 6ms/step - loss: 26669.9863 - r_squared: -327.0475 - val_loss: 24607.8359 - val_r_squared: -303.2839
Epoch 5/300
87/87 [==============================] - 0s 5ms/step - loss: 21990.4980 - r_squared: -279.1728 - val_loss: 19611.3945 - val_r_squared: -241.2243
Epoch 6/300
87/87 [==============================] - 0s 5ms/step - loss: 16846.3906 - r_squared: -215.2191 - val_loss: 14459.9033 - val_r_squared: -177.4470
Epoch 7/300
87/87 [==============================] - 0s 6m

In [84]:
df_test_scaled = scaler.transform(df_test)
y_pred = model.predict(df_test_scaled)

25/25 [==============================] - 0s 1ms/step


In [85]:
predictions_df = pd.DataFrame(y_pred, columns=['Predicted'])

In [86]:
predictions_df['id'] = df_test_id
submission = predictions_df[['id', 'Predicted']]

In [87]:
submission.head()

,id,Predicted
0,0.0,166.537231
1,1.0,171.057495
2,2.0,186.661423
3,3.0,169.930710
4,4.0,174.697845


In [88]:
submission.to_csv('pred_kombinasi3_neural_network.csv', index=False)